__Data preprocessing of the raw train-test data, to remove any duplicates, label conflicts, or 
IC50 value errors__

In [1]:
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import Descriptors, MACCSkeys
from rdkit.ML.Descriptors import MoleculeDescriptors

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate

# Starting data preprocessing

## 1. Read the data

In [2]:
all_data_path = "../../data_for_modeling/raw_data/XO_raw_data.xlsx"
dataset = pd.read_excel(all_data_path, sheet_name='Sheet1')

In [3]:
dataset_c = dataset.copy() #Get a copy of the original dataset
print(len(dataset))
dataset.head()

624


,CID,SMILES,IC50(nM),aid,Positive control substance,IC50_positive (nM),Type,Substructure,Natural products,Plant species,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Code,Substructure.1
0,190,C1=NC2=NC=NC(=C2N1)N,10890.0,287937,Allopurinol,7820.0,active,3,plant,water extract of wheat leaf,NaN,NaN,NaN,NaN,1.0,"1,2,4-Triazole"
1,190,C1=NC2=NC=NC(=C2N1)N,75610.0,1799677,Allopurinol,800.0,active,3,NaN,NaN,NaN,NaN,NaN,NaN,2.0,catechol
2,471,C1=CC(=C(C=C1C2C(C(=O)C3=C(C=C(C=C3O2)O)O)O)O)O,100000.0,399340,Allopurinol,240.0,active,2;14,flavonoids,NaN,NaN,NaN,NaN,NaN,3.0,pyrimidine
3,675,CC1=CC2=C(C=C1C)N=CN2,200000.0,287937,Allopurinol,7820.0,active,16,plant,water extract of wheat leaf,NaN,NaN,NaN,NaN,4.0,1H-pyrazole-4-carbonitrile
4,938,C1=CC(=CN=C1)C(=O)O,518230.0,1444598,Allopurinol,2000.0,active,16,NaN,NaN,NaN,NaN,NaN,NaN,5.0,4-methylthiazole-5-carboxylic acid


In [4]:
ic50_col_name = "IC50(nM)"
activity_col_name = "Type"
smiles_col_name = "SMILES"
cid_col_name = "CID"

In [5]:
def show_activity_distribution(label, dataset):
    #Rows for specific labels
    active_rows = dataset.loc[dataset[label] == "active"]
    inactive_rows = dataset.loc[dataset[label] == "inactive"]
    inconclusive_rows = dataset.loc[dataset[label] == "inconclusive"]
    unspecified_rows = dataset.loc[dataset[label] == "unspecified"]
    dataset_length = len(dataset)
    print("Total dataset")
    table = [['', 'active', 'inactive', 'inconclusive', 'unspecified'], 
            ['Number', len(active_rows), len(inactive_rows), len(inconclusive_rows), len(unspecified_rows)],
            ['Percentage (%)', len(active_rows)/dataset_length*100, len(inactive_rows)/dataset_length*100,
            len(inconclusive_rows)/dataset_length*100, len(unspecified_rows)/dataset_length*100]]
    print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

## 2. Activity distribution

We used to have inconclusive and unspecified labels in the raw data, however, those labels in the raw data have been removed by now.

In [6]:
show_activity_distribution(dataset=dataset, label=activity_col_name)

Total dataset
╒════════════════╤══════════╤════════════╤════════════════╤═══════════════╕
│                │   active │   inactive │   inconclusive │   unspecified │
╞════════════════╪══════════╪════════════╪════════════════╪═══════════════╡
│ Number         │ 329      │   295      │              0 │             0 │
├────────────────┼──────────┼────────────┼────────────────┼───────────────┤
│ Percentage (%) │  52.7244 │    47.2756 │              0 │             0 │
╘════════════════╧══════════╧════════════╧════════════════╧═══════════════╛


## 3. Remove conflict labels data

In [7]:
def check_label_intersection(dataset, col_name):
    active_rows = dataset.loc[dataset[col_name] == "active"]
    inactive_rows = dataset.loc[dataset[col_name] == "inactive"]
    inconclusive_rows = dataset.loc[dataset[col_name] == "inconclusive"]
    unspecified_rows = dataset.loc[dataset[col_name] == "unspecified"]
    
    cid_active = active_rows.loc[:, 'SMILES']
    cid_inactive = inactive_rows.loc[:, 'SMILES']
    cid_incon = inconclusive_rows.loc[:, 'SMILES']
    cid_unspec = unspecified_rows.loc[:, 'SMILES']
    
    ac_inac_cid = np.intersect1d(cid_active, cid_inactive)
    ac_incon_cid = np.intersect1d(cid_active, cid_incon)
    ac_unspec_cid = np.intersect1d(cid_active, cid_unspec)

    inac_incon_cid = np.intersect1d(cid_inactive, cid_incon)
    incon_unspec_cid = np.intersect1d(cid_incon, cid_unspec)
    inac_unspec_cid = np.intersect1d(cid_inactive, cid_unspec)
    
    print("Activity intersection:")
    table = [['Active-Inactive', 'Active-Inconclusive', 'Active-Unspecified', 'Inactive-Inconclusive', 'Inactive-Unspecified', 'Inconclusive-Unspecifid'], 
             [len(ac_inac_cid), len(ac_incon_cid), len(ac_unspec_cid), len(inac_incon_cid), len(inac_unspec_cid), len(incon_unspec_cid)]]
    print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

In [8]:
check_label_intersection(dataset=dataset, col_name=activity_col_name)

Activity intersection:
╒═══════════════════╤═══════════════════════╤══════════════════════╤═════════════════════════╤════════════════════════╤═══════════════════════════╕
│   Active-Inactive │   Active-Inconclusive │   Active-Unspecified │   Inactive-Inconclusive │   Inactive-Unspecified │   Inconclusive-Unspecifid │
╞═══════════════════╪═══════════════════════╪══════════════════════╪═════════════════════════╪════════════════════════╪═══════════════════════════╡
│                 9 │                     0 │                    0 │                       0 │                      0 │                         0 │
╘═══════════════════╧═══════════════════════╧══════════════════════╧═════════════════════════╧════════════════════════╧═══════════════════════════╛


# Remove error data

## Remove IC50 errors data

In [9]:
def find_non_float_ic50(dataset, ic50_col_name):
    # Use to_numeric to identify non-float values
    is_float = pd.to_numeric(dataset[ic50_col_name], errors='coerce').notna()
    # Find the rows where 'IC50 (nM)' is not a float
    non_float_rows = dataset[~is_float]
    return non_float_rows

dataset[ic50_col_name] = pd.to_numeric(dataset[ic50_col_name])
dataset = dataset.dropna(subset=[ic50_col_name])
dataset = dataset.drop(dataset.loc[dataset[ic50_col_name] == 0].index)

In [10]:
len(dataset)

624

In [11]:
show_activity_distribution(dataset=dataset, label=activity_col_name)

Total dataset
╒════════════════╤══════════╤════════════╤════════════════╤═══════════════╕
│                │   active │   inactive │   inconclusive │   unspecified │
╞════════════════╪══════════╪════════════╪════════════════╪═══════════════╡
│ Number         │ 329      │   295      │              0 │             0 │
├────────────────┼──────────┼────────────┼────────────────┼───────────────┤
│ Percentage (%) │  52.7244 │    47.2756 │              0 │             0 │
╘════════════════╧══════════╧════════════╧════════════════╧═══════════════╛


In [12]:
non_float_rows = find_non_float_ic50(dataset=dataset, ic50_col_name=ic50_col_name)
non_float_rows.head()

,CID,SMILES,IC50(nM),aid,Positive control substance,IC50_positive (nM),Type,Substructure,Natural products,Plant species,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Code,Substructure.1


In [13]:
print(len(dataset.loc[dataset[ic50_col_name] == 0].index))

0


In [14]:
data_ic50_processed = dataset.copy()

## Remove labels intersection

__Remove label intersection__

In [15]:
def check_label_intersection(dataset, col_name):
    '''
        Description: Search for any data that have the same SMILES but have different labels
    '''
    active_rows = dataset.loc[dataset[col_name] == "active"]
    inactive_rows = dataset.loc[dataset[col_name] == "inactive"]
    inconclusive_rows = dataset.loc[dataset[col_name] == "inconclusive"]
    unspecified_rows = dataset.loc[dataset[col_name] == "unspecified"]
    
    cid_active = active_rows.loc[:, 'SMILES']
    cid_inactive = inactive_rows.loc[:, 'SMILES']
    cid_incon = inconclusive_rows.loc[:, 'SMILES']
    cid_unspec = unspecified_rows.loc[:, 'SMILES']
    
    ac_inac_cid = np.intersect1d(cid_active, cid_inactive)
    ac_incon_cid = np.intersect1d(cid_active, cid_incon)
    ac_unspec_cid = np.intersect1d(cid_active, cid_unspec)

    inac_incon_cid = np.intersect1d(cid_inactive, cid_incon)
    incon_unspec_cid = np.intersect1d(cid_incon, cid_unspec)
    inac_unspec_cid = np.intersect1d(cid_inactive, cid_unspec)
    
    print("Activity intersection:")
    table = [['Active-Inactive', 'Active-Inconclusive', 'Active-Unspecified', 'Inactive-Inconclusive', 'Inactive-Unspecified', 'Inconclusive-Unspecifid'], 
             [len(ac_inac_cid), len(ac_incon_cid), len(ac_unspec_cid), len(inac_incon_cid), len(inac_unspec_cid), len(incon_unspec_cid)]]
    print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

In [16]:
check_label_intersection(dataset=dataset, col_name=activity_col_name)

Activity intersection:
╒═══════════════════╤═══════════════════════╤══════════════════════╤═════════════════════════╤════════════════════════╤═══════════════════════════╕
│   Active-Inactive │   Active-Inconclusive │   Active-Unspecified │   Inactive-Inconclusive │   Inactive-Unspecified │   Inconclusive-Unspecifid │
╞═══════════════════╪═══════════════════════╪══════════════════════╪═════════════════════════╪════════════════════════╪═══════════════════════════╡
│                 9 │                     0 │                    0 │                       0 │                      0 │                         0 │
╘═══════════════════╧═══════════════════════╧══════════════════════╧═════════════════════════╧════════════════════════╧═══════════════════════════╛


In [40]:
def get_new_data_by_activity(dataset, activity, activity_col_name, new_columns, ic50_col_name):
    #Handling the active rows first
    data_rows = dataset.loc[dataset[activity_col_name] == activity]
    new_rows = pd.DataFrame(columns=new_columns)
    for cid in data_rows.CID.unique():
        sub_dataset = data_rows.loc[data_rows.CID == cid]
        avg_ic50, dup_count = 0, 0
        for index, row in sub_dataset.iterrows():
            avg_ic50 = avg_ic50 + float(row[ic50_col_name])/len(sub_dataset)
            dup_count = dup_count + 1
        new_rows.loc[len(new_rows)] = [row.CID, row.SMILES ,avg_ic50, activity,
                                            {True: 'active', False: 'inactive'} [avg_ic50 < 190],
                                            dup_count]
    return new_rows

In [41]:
new_columns = ["CID", "SMILES", "AVG_IC50_nM", "FIRST_LABEL", "FINAL_LABEL", "DUPLICATE_COUNTS"]

In [42]:
new_active_rows = get_new_data_by_activity(dataset=dataset, activity='active', activity_col_name=activity_col_name, new_columns=new_columns, ic50_col_name=ic50_col_name)
print(len(new_active_rows))
new_active_rows.head()

269


,CID,SMILES,AVG_IC50_nM,FIRST_LABEL,FINAL_LABEL,DUPLICATE_COUNTS
0,190,C1=NC2=NC=NC(=C2N1)N,43250.0,active,inactive,2
1,471,C1=CC(=C(C=C1C2C(C(=O)C3=C(C=C(C=C3O2)O)O)O)O)O,100000.0,active,inactive,1
2,675,CC1=CC2=C(C=C1C)N=CN2,200000.0,active,inactive,1
3,938,C1=CC(=CN=C1)C(=O)O,518230.0,active,inactive,1
4,4947,CCCOC(=O)C1=CC(=C(C(=C1)O)O)O,628000.0,active,inactive,1


__Inactive rows__

In [23]:
new_inactive_rows = get_new_data_by_activity(dataset=dataset, activity='Inactive', ic50_col_name=ic50_col_name, activity_col_name=activity_col_name, new_columns=new_columns)
print(len(new_inactive_rows))
new_inactive_rows

0


,CID,SMILES,AVG_IC50_nM,FIRST_LABEL,FINAL_LABEL,DUPLICATE_COUNTS


__Combine these data__

In [24]:
new_data = pd.DataFrame(columns=new_columns)
new_data = pd.concat([new_data, new_active_rows], axis=0)
new_data = pd.concat([new_data, new_inactive_rows], axis=0)
print(len(new_active_rows), len(new_inactive_rows), len(new_data))

0 0 0


In [25]:
new_data.head()

,CID,SMILES,AVG_IC50_nM,FIRST_LABEL,FINAL_LABEL,DUPLICATE_COUNTS


In [26]:
show_activity_distribution(dataset=new_data, label='FINAL_LABEL')

Total dataset


ZeroDivisionError: division by zero

In [142]:
check_label_intersection(dataset=new_data, col_name='FIRST_LABEL')

Activity intersection:
╒═══════════════════╤═══════════════════════╤══════════════════════╤═════════════════════════╤════════════════════════╤═══════════════════════════╕
│   Active-Inactive │   Active-Inconclusive │   Active-Unspecified │   Inactive-Inconclusive │   Inactive-Unspecified │   Inconclusive-Unspecifid │
╞═══════════════════╪═══════════════════════╪══════════════════════╪═════════════════════════╪════════════════════════╪═══════════════════════════╡
│                 0 │                     0 │                    0 │                       0 │                      0 │                         0 │
╘═══════════════════╧═══════════════════════╧══════════════════════╧═════════════════════════╧════════════════════════╧═══════════════════════════╛


# Remove duplicate smiles

In [143]:
def remove_dup_smiles(dataset, smile_col_name):
    unique_result = pd.DataFrame(columns=dataset.columns)
    dup_result = pd.DataFrame(columns=dataset.columns)
    error_result = pd.DataFrame(columns=dataset.columns)
    #Start iteration
    unique_smiles = dataset[smile_col_name].unique()
    for smile in unique_smiles:
        sub_dataset = dataset.loc[dataset[smile_col_name] == smile]
        if(len(sub_dataset) == 1):
            unique_result = pd.concat([unique_result, sub_dataset], axis=0)
        elif(len(sub_dataset) > 1):
            dup_result = pd.concat([dup_result, sub_dataset], axis=0)
        else:
            error_result = pd.concat([error_result, sub_dataset], axis=0)
    return unique_result, dup_result, error_result

In [144]:
unique_smiles = dataset[smiles_col_name].unique()
len(unique_smiles)

2801

In [145]:
unique_result, dup_result, error_result = remove_dup_smiles(dataset=new_data, smile_col_name=smiles_col_name)

In [146]:
print(len(unique_result), len(dup_result), len(error_result))

2801 0 0


In [147]:
dup_result.head()

,CID,SMILES,AVG_IC50_uM,FIRST_LABEL,FINAL_LABEL,DUPLICATE_COUNTS


In [148]:
new_data = unique_result
print(len(new_data))
new_data.head()

2801


,CID,SMILES,AVG_IC50_uM,FIRST_LABEL,FINAL_LABEL,DUPLICATE_COUNTS
0,3994,CN(C)c1ccc(cc1)C(=O)NCCCCCCC(=O)NO,20.700000,Active,Active,1
1,5311,O=C(CCCCCCC(=O)Nc1ccccc1)NO,189.846512,Active,Active,1
2,65628,Cn1c(CCCC(=O)O)nc2cc(N(CCCl)CCCl)ccc21,9.500000,Active,Active,1
3,4596836,CCCCNNC(=O)c1ccc(Br)cc1,155.500000,Active,Active,1
4,6400741,O=C(NCCSSCCNC(=O)/C(Cc1ccc(O)c(Br)c1)=N/O)/C(C...,33.000000,Active,Active,1


In [149]:
dup_result.head()

,CID,SMILES,AVG_IC50_uM,FIRST_LABEL,FINAL_LABEL,DUPLICATE_COUNTS


# Check the preprocessed dataset again

In [150]:
check_label_intersection(dataset=new_data, col_name='FIRST_LABEL')

Activity intersection:
╒═══════════════════╤═══════════════════════╤══════════════════════╤═════════════════════════╤════════════════════════╤═══════════════════════════╕
│   Active-Inactive │   Active-Inconclusive │   Active-Unspecified │   Inactive-Inconclusive │   Inactive-Unspecified │   Inconclusive-Unspecifid │
╞═══════════════════╪═══════════════════════╪══════════════════════╪═════════════════════════╪════════════════════════╪═══════════════════════════╡
│                 0 │                     0 │                    0 │                       0 │                      0 │                         0 │
╘═══════════════════╧═══════════════════════╧══════════════════════╧═════════════════════════╧════════════════════════╧═══════════════════════════╛


In [151]:
check_label_intersection(dataset=new_data, col_name='FINAL_LABEL')

Activity intersection:
╒═══════════════════╤═══════════════════════╤══════════════════════╤═════════════════════════╤════════════════════════╤═══════════════════════════╕
│   Active-Inactive │   Active-Inconclusive │   Active-Unspecified │   Inactive-Inconclusive │   Inactive-Unspecified │   Inconclusive-Unspecifid │
╞═══════════════════╪═══════════════════════╪══════════════════════╪═════════════════════════╪════════════════════════╪═══════════════════════════╡
│                 0 │                     0 │                    0 │                       0 │                      0 │                         0 │
╘═══════════════════╧═══════════════════════╧══════════════════════╧═════════════════════════╧════════════════════════╧═══════════════════════════╛


In [152]:
print("Data size: " + str(len(new_data)))
print("Label for first label:")
show_activity_distribution(dataset=new_data, label='FIRST_LABEL')
print()
print("Label for final label:")
show_activity_distribution(dataset=new_data, label='FINAL_LABEL')

Data size: 2801
Label for first label:
Total dataset
╒════════════════╤══════════╤════════════╤════════════════╤═══════════════╕
│                │   Active │   Inactive │   Inconclusive │   Unspecified │
╞════════════════╪══════════╪════════════╪════════════════╪═══════════════╡
│ Number         │ 899      │  1902      │              0 │             0 │
├────────────────┼──────────┼────────────┼────────────────┼───────────────┤
│ Percentage (%) │  32.0957 │    67.9043 │              0 │             0 │
╘════════════════╧══════════╧════════════╧════════════════╧═══════════════╛

Label for final label:
Total dataset
╒════════════════╤══════════╤════════════╤════════════════╤═══════════════╕
│                │   Active │   Inactive │   Inconclusive │   Unspecified │
╞════════════════╪══════════╪════════════╪════════════════╪═══════════════╡
│ Number         │ 899      │  1902      │              0 │             0 │
├────────────────┼──────────┼────────────┼────────────────┼──────────────

In [153]:
non_float_rows = find_non_float_ic50(dataset, ic50_col_name=ic50_col_name)
print(len(non_float_rows))
non_float_rows.head()

0


,CID,CHEMBL_ID,BDB_ID,SMILES,IC50 CHEMBL,IC50 BDMB,IC50_nM,LABELS,ZBG Classified


__Check if the SMILES are encodedable__

In [154]:
from tqdm import tqdm

def maccs_fpts(data):
    Maccs_fpts = []
    error_index = []
    count = 0
    with tqdm(total=len(data), desc='Progress') as pbar:
        for i in data:
            try:
                mol = Chem.MolFromSmiles(i)
                fpts = MACCSkeys.GenMACCSKeys(mol)
            except:
                print("An exception occurred with " + str(count))
                error_index.append(count)
                count += 1
                continue
            mfpts = np.array(fpts)
            Maccs_fpts.append(mfpts)
            count += 1
            pbar.update(1)  # Update the progress bar
    return np.array(Maccs_fpts), error_index

In [155]:
smiles_data, error_idx = maccs_fpts(new_data['SMILES'])

Progress: 100%|██████████| 2801/2801 [00:01<00:00, 1420.71it/s]


In [163]:
new_data.head()

,CID,SMILES,AVG_IC50_uM,FIRST_LABEL,FINAL_LABEL,DUPLICATE_COUNTS
0,3994,CN(C)c1ccc(cc1)C(=O)NCCCCCCC(=O)NO,20.700000,Active,Active,1
1,5311,O=C(CCCCCCC(=O)Nc1ccccc1)NO,189.846512,Active,Active,1
2,65628,Cn1c(CCCC(=O)O)nc2cc(N(CCCl)CCCl)ccc21,9.500000,Active,Active,1
3,4596836,CCCCNNC(=O)c1ccc(Br)cc1,155.500000,Active,Active,1
4,6400741,O=C(NCCSSCCNC(=O)/C(Cc1ccc(O)c(Br)c1)=N/O)/C(C...,33.000000,Active,Active,1


In [161]:
dataset_c.head()

,CID,CHEMBL_ID,BDB_ID,SMILES,IC50 CHEMBL,IC50 BDMB,IC50_nM,LABELS,ZBG Classified
0,264,CHEMBL14227,26109.0,CCCC(=O)O,206000.000000,206000,206000.000000,Inactive,3
1,2746,CHEMBL235191,19422.0,CC(=O)Nc1ccc(C(=O)Nc2ccccc2N)cc1,1495.875000,1040.083333,1495.875000,Inactive,4
2,2788,NaN,32188.0,Oc1c(I)cc(Cl)c2cccnc12,NaN,105800,105800.000000,Inactive,21
3,3121,CHEMBL109,50003616.0,CCCC(CCC)C(=O)O,288246.666667,288246.666667,288246.666667,Inactive,3
4,3994,NaN,50082665.0,CN(C)c1ccc(cc1)C(=O)NCCCCCCC(=O)NO,NaN,20.7,20.700000,Active,1


__Write to file__

In [164]:
with pd.ExcelWriter('../../data_for_modeling/preprocessed_data/HDAC2_all_data_filtered_p2.xlsx', engine='openpyxl') as writer:
    writer.book = writer.book 
    dataset_c.to_excel(writer, sheet_name='original_data', index=False)
    new_data.to_excel(writer, sheet_name='filter_data', index=False)
    dup_result.to_excel(writer, sheet_name='duplicate_smiles', index=False)
    data_ic50_processed.to_excel(writer, sheet_name="data_ic50_process", index=False)

/tmp/ipykernel_191960/3127742887.py:2: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = writer.book
